### Import Packages

In [1]:
import pickle5 as pickle
pic_='F'
tracking_uri_='http://34.128.104.38:5000'

### 01 Hyper Parameter Tuning for Catboost Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [ ]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_a_Catboost-Optuna-MLFlow [No Artificial].py'
%run -i '05_a_Catboost-Optuna-MLFlow [With Artificial].py'
print('DONE')
print('==============')

### 02 Hyper Parameter Tuning for LightGBM Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [ ]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_b_LightGBM-Optuna-MLFlow [No Artifcial].py'
%run -i '05_b_LightGBM-Optuna-MLFlow [With Artifcial].py'
print('DONE')
print('==============')

### 03 Hyper Parameter Tuning for RandomForest Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [ ]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_c_RandomForest-Optuna-MLFlow [No Artifcial].py'
%run -i '05_c_RandomForest-Optuna-MLFlow [With Artifcial].py'
print('DONE')
print('==============')

### 04 Comparison Model and Bundle the Result into Pickle
this code is performed in both artificial (Syntetic) and non artificial data.

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
model_tuning={}

In [3]:
client = MlflowClient()
experiment = client.get_experiment_by_name("Hyper Parameter Tuning")
if experiment!=None:
    exp_id=experiment.experiment_id
    data_=mlflow.search_runs([exp_id], order_by=["metrics.f1_micro DESC","metrics.logloss ASC","metrics.roc_auc_score DESC"])

In [4]:
best_cat_no_artificial=data_.loc[(data_['params.algorithm']=='catboost')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_cat_no_artificial='runs:/'+best_cat_no_artificial+'/catboost_model'
best_cat_no_artificial=mlflow.catboost.load_model(best_cat_no_artificial)

best_cat_w_artificial=data_.loc[(data_['params.algorithm']=='catboost')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_cat_w_artificial='runs:/'+best_cat_w_artificial+'/catboost_model'
best_cat_w_artificial=mlflow.catboost.load_model(best_cat_w_artificial)


best_lgb_no_artificial=data_.loc[(data_['params.algorithm']=='lightgbm')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_lgb_no_artificial='runs:/'+best_lgb_no_artificial+'/lgbm_model'
best_lgb_no_artificial=mlflow.lightgbm.load_model(best_lgb_no_artificial)

best_lgb_w_artificial=data_.loc[(data_['params.algorithm']=='lightgbm')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_lgb_w_artificial='runs:/'+best_lgb_w_artificial+'/lgbm_model'
best_lgb_w_artificial=mlflow.lightgbm.load_model(best_lgb_w_artificial)


best_rf_no_artificial=data_.loc[(data_['params.algorithm']=='randomforest')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_rf_no_artificial='runs:/'+best_rf_no_artificial+'/rf_model'
best_rf_no_artificial=mlflow.sklearn.load_model(best_rf_no_artificial)

best_rf_w_artificial=data_.loc[(data_['params.algorithm']=='randomforest')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_rf_w_artificial='runs:/'+best_rf_w_artificial+'/rf_model'
best_rf_w_artificial=mlflow.sklearn.load_model(best_rf_w_artificial)

In [5]:
model_tuning['catboost_wo_na']=best_cat_no_artificial
model_tuning['lightgbm_wo_na']=best_lgb_no_artificial
model_tuning['randomforest_wo_na']=best_rf_no_artificial

model_tuning['catboost_w_a']=best_cat_w_artificial
model_tuning['lightgbm_w_a']=best_lgb_w_artificial
model_tuning['randomforest_w_a']=best_rf_w_artificial

### 05 Export Data
this code is performed in both artificial (Syntetic) and non artificial data.

In [6]:
with open('ml_output/04_05_modeling/tuning/model_tuned.pickle', 'wb') as handle:
    pickle.dump(model_tuning, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
import os
from google.cloud import storage
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'config/gcp_store.json'
bucket = client.get_bucket('bps-gcp-bucket')
file_name='MLST2023/model/tuning-model/'+pic_+'_tuning.pickle'
bucket.blob(file_name).upload_from_filename('ml_output/05_trials_hypertuning/model_tuned.pickle')